002は途中まで動かせるが、エラーが起きるので、003で002と同ステップの内容を実施することとした

## 1. 必要なライブラリのインポート

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from transformers import AutoModel, AutoTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

## 2. データセットの準備とクライアントへの分割 (変更)
フェデレーテッド学習をシミュレートするため、ユーザーを複数の「クライアント」に分割します。各クライアントは自身のユーザーIDとインタラクションデータを持つことになります。

In [ ]:
num_users = 100
num_items = 50
num_clients = num_users # NOTE: 普通、1クライアントが複数のユーザを持つことがあるので、ユーザとクライアントを別で定義する.


# ユーザーのテキスト特徴 (例: 趣味、自己紹介など)
user_texts = {i: f"This user likes movies about {i % 5} and enjoys {i % 3}." for i in range(num_users)}

# ユーザーアイテムインタラクション (implicit feedback)
interactions_list = []
for u_id in range(num_users):
    for i_id in range(num_items):
        if np.random.rand() > 0.7:  # 約30%の確率でインタラクションあり
            interactions_list.append([u_id, i_id, 1])
        else:
            interactions_list.append([u_id, i_id, 0])

interactions = torch.tensor(interactions_list, dtype=torch.float32)

# 各クライアントにユーザを割り当てる
client_user_map = defaultdict(list)
for u_id in range(num_users):
    client_id = u_id % num_clients
    client_user_map[client_id].append(u_id)

# for key, val in client_user_map.items():
#     print(f"Client {key} has users: {val}")

client_datasets = {}
for client_id, uids in client_user_map.items():
    # 各クライアントのインタラクションデータを抽出
    client_interactions_indices = [i for i, (u,_,_) in enumerate(interactions_list) if u in uids]
    client_subset = Subset(
        TensorDataset(
            interactions[:,0].long(),
            interactions[:,1].long(),
            interactions[:,2].float()
        ),
        client_interactions_indices
    )
    client_datasets[client_id] = DataLoader(client_subset, batch_size=32, shuffle=True)

# for client_id, dataset in client_datasets.items():
#     print(f"Client {client_id} dataset size: {len(dataset)}")

Client 0 has users: [0]
Client 1 has users: [1]
Client 2 has users: [2]
Client 3 has users: [3]
Client 4 has users: [4]
Client 5 has users: [5]
Client 6 has users: [6]
Client 7 has users: [7]
Client 8 has users: [8]
Client 9 has users: [9]
Client 10 has users: [10]
Client 11 has users: [11]
Client 12 has users: [12]
Client 13 has users: [13]
Client 14 has users: [14]
Client 15 has users: [15]
Client 16 has users: [16]
Client 17 has users: [17]
Client 18 has users: [18]
Client 19 has users: [19]
Client 20 has users: [20]
Client 21 has users: [21]
Client 22 has users: [22]
Client 23 has users: [23]
Client 24 has users: [24]
Client 25 has users: [25]
Client 26 has users: [26]
Client 27 has users: [27]
Client 28 has users: [28]
Client 29 has users: [29]
Client 30 has users: [30]
Client 31 has users: [31]
Client 32 has users: [32]
Client 33 has users: [33]
Client 34 has users: [34]
Client 35 has users: [35]
Client 36 has users: [36]
Client 37 has users: [37]
Client 38 has users: [38]
Client

## 3. 軽量 LLM 埋め込みモデルのロード

In [22]:
# Hugging Face の軽量 LLM 埋め込みモデルのロード
plm_model_name = "sentence-transformers/all-MiniLM-L6-v2"
plm_tokenizer = AutoTokenizer.from_pretrained(plm_model_name)
plm_model = AutoModel.from_pretrained(plm_model_name)

# PLMは学習済みモデルのため、勾配計算を無効化
for param in plm_model.parameters():
    param.requires_grad = False

plm_embedding_dim = plm_model.config.hidden_size
print(f"PLM embedding dimension: {plm_embedding_dim}")

PLM embedding dimension: 384


## 4. モデルの定義 (変更あり: グローバルアイテム埋め込みの追加)
各クライアントが持つモデルに加えて、サーバーが管理する「グローバルアイテム埋め込み」を導入します。これは、論文における 

theta_item に相当します 。

In [23]:
# クライアント側のモデルを定義して、その後サーバ側を定義する

In [24]:
# クライアントのモデル
class ClientModel(nn.Module):
    def __init__(
        self,
        num_items,
        item_embedding_dim,
        plm_model,
        plm_embedding_dim,
        joint_embedding_output_dim,
    ):
        super(ClientModel, self).__init__()
        self.plm_model = plm_model # NOTE: PLMは全クライアントで同じものを利用する.重みは固定である.

        # joint embedding layer
        self.user_joint_embedding_linear = nn.Linear(plm_embedding_dim, joint_embedding_output_dim)

        # item embedding layer
        self.local_item_embedding = nn.Embedding(num_items, item_embedding_dim)

        # 予測層 (各クライアントのローカルパラメータ)
        self.prediction_layer = nn.Linear(joint_embedding_output_dim + item_embedding_dim, 1)

    def forward(self, user_ids, item_ids, user_texts_batch):
        encoded_input = plm_tokenizer(user_texts_batch, return_tensors='pt', padding=True, truncation=True)
        plm_output = self.plm_model(**encoded_input).last_hidden_state[:, 0, :] # [CLS]トークンの埋め込みを使用

        # Joint Embedding Layer の線形変換
        user_embedding = self.user_joint_embedding_linear(plm_output) # (batch_size, joint_embedding_output_dim)

        # ローカルのアイテム埋め込み
        item_embedding = self.local_item_embedding(item_ids) # (batch_size, item_embedding_dim)

        # ユーザー埋め込みとアイテム埋め込みを結合
        combined_features = torch.cat((user_embedding, item_embedding), dim=1)

        # 予測
        prediction = torch.sigmoid(self.prediction_layer(combined_features))
        return prediction, self.user_joint_embedding_linear.weight, item_embedding # 線形層の重みとアイテム埋め込みを返す


In [ ]:
# サーバのモデル
class Server:
    def __init__(
        self,
        num_users,
        num_items,
        item_embedding_dim,
        joint_embedding_output_dim,
    ):
        self.num_users = num_users
        self.num_items = num_items
        self.item_embedding_dim = item_embedding_dim
        self.joint_embedding_output_dim = joint_embedding_output_dim
        self.global_item_embedding = nn.Embedding(num_items, item_embedding_dim)

        # # グローバルアイテム埋め込みのパラメータを学習対象とする
        # self.optimizer = optim.Adam(self.global_item_embedding.parameters(), lr=0.001)

    def build_user_relationship_graph(
        self,
        client_user_linear_weights,
        user_ids_in_client,
    ):
        """
        ユーザーのジョイント埋め込み層の重み行列からユーザー関係グラフを構築します。
        論文の式 (15) に基づいています [cite: 105, 106]。
        """
        all_user_weights = {}

        for client_id, weight_matrix in client_user_linear_weights.items():
            for i, user_id in enumerate(user_ids_in_client[client_id]):
                # 重み行列をベクトル化
                all_user_weights[user_id] = weight_matrix[i].flatten().detach().cpu().numpy()

        # 全ユーザーの重みベクトルを収集
        sorted_user_ids = sorted(all_user_weights.keys())
        if not sorted_user_ids: # ユーザーがいない場合
            return np.zeros((self.num_users, self.num_users))

        user_weight_vectors = np.array([all_user_weights[uid] for uid in sorted_user_ids])

        # コサイン類似度で類似度行列を計算
        similarity_matrix = cosine_similarity(user_weight_vectors)

        # 各ユーザーの上位N個の類似ユーザーを選択してグラフを構築 (ここでは簡単のため、全てのユーザー間の類似度を使用)
        # 論文のStep2では「top-N in the highest similarity list」とあるが [cite: 108]、
        # ここでは完全な類似度グラフ (隣接行列) を使用。
        # 厳密には、ここで閾値を設けるか、上位K個のみを選択して疎なグラフを構築すべき。
        # S' に相当 [cite: 108]
        user_graph_adj = similarity_matrix

        return user_graph_adj, sorted_user_ids

    def aggregate_item_embeddings(
        self,
        client_item_embeddings,
        user_graph_adj,
        sorted_user_ids,
    ):
        """
        ユーザー関係グラフに基づいて、アイテム埋め込みをグローバルに集約します。
        論文の式 (16) と (17)  に基づいています。

        Args:
            client_local_item_weights (dict):
            user_graph_adj (np.ndarray): ユーザーグラフの隣接行列
            sorted_user_ids (list): user_graph_adj のノード順に対応するクライアントIDのリスト

        Returns:
            torch.Tensor: 更新されたグローバルアイテム埋め込みの重み
        """
        # 各ユーザの現在のアイテム埋め込みを収集する（ユーザID順に並べ替える）
        current_item_embeddings_map = {}
        for client_id, item_embs in client_item_embeddings.items():
            # クライアントが処理したユーザIDと対応するアイテム埋め込み
            for u_id, item_emb in item_embs.items():
                # 各ユーザーが複数のアイテムとインタラクションする可能性があるため、
                # ここでは簡略化のため、各ユーザーのアイテム埋め込みの平均を使用する
                # 論文では「the I-th row represents the item embedding obtained from user i」
                # とあり、ユーザーiが学習したアイテム埋め込み全体を指す。
                # ここでは、そのユーザーが学習したアイテム埋め込みの平均を代表値とする。
                current_item_embeddings_map[u_id] = item_emb.mean(dim=0).detach().cpu().numpy()

        # グラフの順序に合わせてアイテム埋め込みを行列にまとめる
        item_embedding_matrix_A = np.array([current_item_embeddings_map[uid] for uid in sorted_user_ids])

        # グラフ畳み込み (LightGCNの簡略版) [cite: 110, 111]
        # 論文の式(16): R = S'' A  (S''は正規化された隣接行列)
        # ここではS''を正規化された user_graph_adj とする
        # 正規化: D^{-1/2} A D^{-1/2} (ただし、ここでは簡易的に行和で正規化)
        # 厳密には LightGCN の伝播ルールに従うべきだが、ここでは単純化

        # 簡単な正規化
        row_sums = user_graph_adj.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1 # ゼロ除算を避ける
        normalized_adj = user_graph_adj / row_sums

        # グラフ畳み込み
        # R が学習後の相関行列
        R = np.dot(normalized_adj, item_embedding_matrix_A)

        # グローバルアイテム埋め込みの更新 (論文の式(17)に基づく) [cite: 113, 114]
        # θ_global = DR  (Dはaggregation時のdegree matrix [cite: 114])
        # ここでは、Rの各行（ユーザーiによって学習されたアイテム埋め込み）の平均をとることで、
        # グローバルなアイテム埋め込みを導出すると解釈する。
        # これは FedAvg に近いシンプルな集約方法。
        # 厳密なDは、各ユーザーの貢献度に応じた重み付け行列だが、
        # まずは単純平均で実装する。
        new_global_item_embedding_np = np.mean(R, axis=0)

        # NumPy配列をPyTorchテンソルに変換し、グローバルアイテム埋め込みを更新
        new_global_item_embedding_tensor = torch.tensor(new_global_item_embedding_np, dtype=torch.float32)

        # サーバーのグローバルアイテム埋め込みを直接更新
        self.global_item_embedding.weight.data.copy_(new_global_item_embedding_tensor)
        return self.global_item_embedding.weight.data # 更新されたグローバルアイテム埋め込みの重みを返す

## 5. 学習ループの変更 (フェデレーテッド学習のシミュレーション)
この学習ループでは、以下のステップをシミュレートします。

グローバル配布: サーバーがグローバルアイテム埋め込みを各クライアントに配布します。

ローカル学習: 各クライアントは自身のデータを使ってモデルを学習し、ユーザーのジョイント埋め込み層の重みと、ローカルで更新されたアイテム埋め込み（のテンソル）をサーバーにアップロードします。

グラフ集約: サーバーはアップロードされたユーザー埋め込み層の重みからユーザー関係グラフを構築し、それに基づいてグローバルアイテム埋め込みを集約・更新します。

In [ ]:
item_embedding_dim = 32
joint_embedding_output_dim = 64

# サーバーのインスタンス化
server = Server(num_users, num_items, item_embedding_dim, joint_embedding_output_dim)

# 各クライアントのモデルを辞書で保持
client_models = {}
client_optimizers = {}
for client_id in range(num_clients):
    client_models[client_id] = ClientModel(
        num_items,
        item_embedding_dim,
        plm_model,
        plm_embedding_dim,
        joint_embedding_output_dim
    )
    # NOTE:
    # クライアントごとに最適化するパラメータを設定
    # ここでは、user_joint_embedding_linear, local_item_embedding, prediction_layer が対象
    # 単純にoptim.Adam(params = client_models[client_id].parameters(), lr=0.001)とすると、
    # PLMも学習可能パラメータとなってしまうので、
    # PLMのパラメータを除外したパラメータのみを取得してから、設定する.
    trainable_params = [
        p for name, p in client_models[client_id].named_parameters()
        if not name.startswith('plm_model.')
    ]

    client_optimizers[client_id] = optim.Adam(
        params=trainable_params,
        lr=0.001
    )


# 学習ループ (フェデレーテッド学習ラウンド)
num_communication_rounds = 10
local_epochs = 1 # 各クライアントのローカル学習エポック数


for round_num in range(num_communication_rounds):
    print(f"\n--- Communication Round {round_num + 1}/{num_communication_rounds} ---")

    # サーバーからグローバルアイテム埋め込みをクライアントに配布
    # (ClientModelのlocal_item_embeddingをサーバーのglobal_item_embeddingで初期化)
    # 論文の「Global Distribution: Updated global project embeddings are broadcast 
    # to all clients for next-round initialization.」[cite: 63]
    for client_id in range(num_clients):
        client_models[client_id].local_item_embedding.weight.data.copy_(server.global_item_embedding.weight.data)

    client_user_linear_weights_to_server = {} # {client_id: user_linear_weight_matrix_from_client}
    client_item_embeddings_to_server = defaultdict(dict) # {client_id: {user_id: item_embeddings_tensor_from_user}}

    # クライアントのローカル学習
    for client_id in range(num_clients):
        model = client_models[client_id]
        optimizer = client_optimizers[client_id]
        dataloader = client_datasets[client_id]

        model.train()
        local_loss = 0.0

        # 各ユーザがどのクライアントに属しているかを把握
        users_in_current_client = client_user_map[client_id]

        for epoch in range(local_epochs):
            for user_ids_batch, item_ids_batch, labels_batch in dataloader:

                # バッチ内のユーザーIDに対応するテキスト特徴を取得
                current_user_texts = [user_texts[uid.item()] for uid in user_ids_batch]

                optimizer.zero_grad()
                predictions, user_linear_weight, item_embs_batch = model(
                    user_ids_batch,
                    item_ids_batch,
                    current_user_texts
                )

                # 損失計算 (論文の式(12) L_all = L_1 + lambda * R) [cite: 95]
                # ここでは簡略化のためL_1のみを使用。Rはまだ導入していない。
                loss = nn.BCELoss()(predictions.squeeze(), labels_batch) # NOTE: ClientModel側の出力ですでにsigmoidを適用しているため、BCELossを使用

                loss.backward()
                optimizer.step()
                local_loss += loss.item()

                # 各ユーザーの線形層の出力 (埋め込み) を収集
                for i, u_id_tensor in enumerate(user_ids_batch):
                    u_id = u_id_tensor.item()
                    # 各ユーザーの線形層の重み (今回は線形層の出力) をクライアントがサーバーに報告する
                    client_user_linear_weights_to_server[client_id].append(user_linear_weight[i].detach().cpu().numpy())

                    # 各ユーザーが学習したアイテム埋め込みも収集 (ここではバッチ内の平均)
                    # 論文の「item embedding obtained from user i」を模倣
                    # ここでは、バッチ内の各ユーザーのアイテム埋め込みの平均として保存
                    client_item_embeddings_to_server[client_id][u_id] = item_embs_batch[i].detach().cpu()
        print(f"  Client {client_id} local loss: {local_loss / len(dataloader):.4f}")

        # ユーザーIDとそれに紐づくテキスト埋め込みを統合する
        all_user_embedding_outputs = {}
        for client_id, user_embs_list in client_user_linear_weights_to_server.items():
            # 各クライアントがアップロードする `user_joint_embedding_linear.weight` を格納する
            client_linear_weights_for_graph = {
                client_id: client_models[client_id].user_joint_embedding_linear.weight.data.clone().flatten()
                for client_id in range(num_clients)
            }


--- Communication Round 1/10 ---
torch.Size([50, 32])
torch.Size([50, 32])


とりあえず

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from transformers import AutoModel, AutoTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# 軽量 LLM 埋め込みモデルのロード (変更なし)
plm_model_name = "sentence-transformers/all-MiniLM-L6-v2"
plm_tokenizer = AutoTokenizer.from_pretrained(plm_model_name)
plm_model = AutoModel.from_pretrained(plm_model_name)

# PLMは学習済みモデルのため、勾配計算を無効化
for param in plm_model.parameters():
    param.requires_grad = False

plm_embedding_dim = plm_model.config.hidden_size
print(f"PLM embedding dimension: {plm_embedding_dim}")


class ClientModel(nn.Module):
    def __init__(self, num_items, item_embedding_dim, plm_model, plm_embedding_dim, joint_embedding_output_dim):
        super(ClientModel, self).__init__()
        self.plm_model = plm_model
        
        # Joint Embedding Layer の線形変換部分 (ユーザーのローカルパラメータ)
        self.user_joint_embedding_linear = nn.Linear(plm_embedding_dim, joint_embedding_output_dim)
        
        # アイテム埋め込み (各クライアントのローカルパラメータとして初期化される)
        self.local_item_embedding = nn.Embedding(num_items, item_embedding_dim)

        # 予測層 (各クライアントのローカルパラメータ)
        self.prediction_layer = nn.Linear(joint_embedding_output_dim + item_embedding_dim, 1)

    def forward(self, user_ids, item_ids, user_texts_batch):
        # ユーザーのテキスト特徴をPLMで埋め込み
        encoded_input = plm_tokenizer(user_texts_batch, padding=True, truncation=True, return_tensors='pt')
        plm_output = self.plm_model(**encoded_input).last_hidden_state[:, 0, :]

        # Joint Embedding Layer の線形変換
        user_embedding = self.user_joint_embedding_linear(plm_output)

        # ローカルのアイテム埋め込み
        item_embedding = self.local_item_embedding(item_ids)

        # ユーザー埋め込みとアイテム埋め込みを結合
        combined_features = torch.cat((user_embedding, item_embedding), dim=1)

        # 予測
        prediction = torch.sigmoid(self.prediction_layer(combined_features))
        return prediction, self.user_joint_embedding_linear.weight, item_embedding # 線形層の重みとアイテム埋め込みを返す


class Server:
    def __init__(self, num_users, num_items, item_embedding_dim, joint_embedding_output_dim):
        # グローバルアイテム埋め込みは、全アイテムの埋め込み行列として管理
        self.global_item_embedding = nn.Embedding(num_items, item_embedding_dim)
        self.num_users = num_users
        self.num_items = num_items
        self.item_embedding_dim = item_embedding_dim
        self.joint_embedding_output_dim = joint_embedding_output_dim
    
    def build_user_relationship_graph(self, client_linear_weights_map):
        """
        各クライアントのuser_joint_embedding_linear.weightからユーザー関係グラフを構築します。
        論文の式 (15)  に基づいています。
        
        Args:
            client_linear_weights_map (dict): {client_id: user_joint_embedding_linear.weight.data (flattened)}
        
        Returns:
            np.ndarray: ユーザーグラフの隣接行列 (NumPy配列)
            list: グラフのノード順に対応するクライアントIDのリスト
        """
        # クライアントIDをユーザーIDとしてグラフを構築
        sorted_client_ids = sorted(client_linear_weights_map.keys())
        if not sorted_client_ids:
            return np.zeros((0, 0)), []

        # クライアントの線形層の重みベクトルを収集
        client_weight_vectors = np.array([
            client_linear_weights_map[c_id].cpu().numpy() for c_id in sorted_client_ids
        ])

        # コサイン類似度で類似度行列を計算 (S_ij) 
        similarity_matrix = cosine_similarity(client_weight_vectors)

        # ここでは簡単のため、完全な類似度グラフを使用 (S' に相当) 
        # 論文の「top-N in the highest similarity list」 は、後のステップで実装可能
        user_graph_adj = similarity_matrix 
        
        return user_graph_adj, sorted_client_ids

    def aggregate_item_embeddings(self, client_local_item_weights, user_graph_adj, sorted_client_ids):
        """
        ユーザー関係グラフに基づいて、アイテム埋め込みをグローバルに集約します。
        論文の式 (16) と (17)  に基づいています。
        
        Args:
            client_local_item_weights (dict): {client_id: local_item_embedding.weight.data (Tensor)}
            user_graph_adj (np.ndarray): ユーザーグラフの隣接行列
            sorted_client_ids (list): user_graph_adj のノード順に対応するクライアントIDのリスト
            
        Returns:
            torch.Tensor: 更新されたグローバルアイテム埋め込みの重み
        """
        if not client_local_item_weights:
            return self.global_item_embedding.weight.data

        # グラフの順序に合わせてクライアントのアイテム埋め込みを行列Aとしてまとめる
        # A は (num_clients, num_items, item_embedding_dim)
        # 論文の「A is the round item embedding matrix, the I-th row represents the item embedding obtained from user i」
        # ここでは、「user i」を「client i」と解釈し、各クライアントの全アイテムの埋め込み行列をAの要素とする
        client_item_embedding_matrix_A = torch.stack([
            client_local_item_weights[c_id] for c_id in sorted_client_ids
        ]) # (num_clients, num_items, item_embedding_dim)

        # グラフの正規化 (S'') 
        # 簡単な行和正規化を使用 (LightGCNの対称正規化とは異なるが、開始点として)
        row_sums_graph = np.sum(user_graph_adj, axis=1, keepdims=True)
        row_sums_graph[row_sums_graph == 0] = 1 # ゼロ除算回避
        normalized_user_graph_adj = user_graph_adj / row_sums_graph
        
        # NumPyをPyTorchテンソルに変換
        normalized_user_graph_adj_tensor = torch.tensor(normalized_user_graph_adj, dtype=torch.float32)

        # グラフ畳み込み (R = S'' A) 
        # R は (num_clients, num_items, item_embedding_dim) となる
        # MatMul: (num_clients, num_clients) x (num_clients, num_items, item_embedding_dim)
        # 結果として、各クライアント（行）が、類似するクライアントのアイテム埋め込みを加重平均したものを得る
        # R_tensor = torch.matmul(normalized_user_graph_adj_tensor, client_item_embedding_matrix_A)
        R_tensor = torch.einsum('ij, jkd -> ikd', normalized_user_graph_adj_tensor, client_item_embedding_matrix_A)

        # グローバルアイテム埋め込みの更新 (θ_global = DR) 
        # 論文のDはAggregation時のdegree matrixだが、ここではRの平均を取ることで、
        # 全体としての共通のアイテム埋め込みを導出すると解釈
        new_global_item_embedding_weight = R_tensor.mean(dim=0) # (num_items, item_embedding_dim)

        # サーバーのグローバルアイテム埋め込みを直接更新
        self.global_item_embedding.weight.data.copy_(new_global_item_embedding_weight)
        
        # 関数自体は新しい埋め込みの値を返すのではなく、内部状態を更新する
        return self.global_item_embedding.weight.data # 更新されたグローバルアイテム埋め込みの重みを返す


# データセットの準備とクライアントへの分割 (変更なし)
num_users = 100
num_items = 50
num_clients = 10

user_texts = {i: f"This user likes movies about {i % 5} and enjoys {i % 3}." for i in range(num_users)}

interactions_list = []
for u_id in range(num_users):
    for i_id in range(num_items):
        if np.random.rand() > 0.7:
            interactions_list.append([u_id, i_id, 1])
        else:
            interactions_list.append([u_id, i_id, 0])

interactions = torch.tensor(interactions_list, dtype=torch.float32)

client_user_map = defaultdict(list)
for u_id in range(num_users):
    client_id = u_id % num_clients
    client_user_map[client_id].append(u_id)

client_datasets = {}
for client_id, uids in client_user_map.items():
    client_interactions_indices = [i for i, (u, _, _) in enumerate(interactions_list) if u in uids]
    client_subset = Subset(TensorDataset(interactions[:, 0].long(), interactions[:, 1].long(), interactions[:, 2]), client_interactions_indices)
    client_datasets[client_id] = DataLoader(client_subset, batch_size=32, shuffle=True)

print(f"Number of users: {num_users}")
print(f"Number of items: {num_items}")
print(f"Number of interactions: {len(interactions)}")
print(f"Number of clients: {num_clients}")


# モデルのハイパーパラメータ
item_embedding_dim = 32
joint_embedding_output_dim = 32 

# サーバーのインスタンス化
server = Server(num_users, num_items, item_embedding_dim, joint_embedding_output_dim)

# 各クライアントのモデルを辞書で保持
client_models = {}
client_optimizers = {}
for client_id in range(num_clients):
    client_models[client_id] = ClientModel(num_items, item_embedding_dim, plm_model, plm_embedding_dim, joint_embedding_output_dim)
    client_optimizers[client_id] = optim.Adam(client_models[client_id].parameters(), lr=0.001)

# 学習ループ (フェデレーテッド学習ラウンド)
num_communication_rounds = 10
local_epochs = 1 

for round_num in range(num_communication_rounds):
    print(f"\n--- Communication Round {round_num + 1}/{num_communication_rounds} ---")
    
    # サーバーからグローバルアイテム埋め込みをクライアントに配布
    for client_id in range(num_clients):
        client_models[client_id].local_item_embedding.weight.data.copy_(server.global_item_embedding.weight.data)

    client_linear_weights_for_graph = {} # {client_id: user_joint_embedding_linear.weight.data (flattened)}
    client_local_item_weights_to_server = {} # {client_id: local_item_embedding.weight.data (Tensor)}
    
    # クライアントのローカル学習
    for client_id in range(num_clients):
        model = client_models[client_id]
        optimizer = client_optimizers[client_id]
        dataloader = client_datasets[client_id]
        
        model.train()
        local_loss = 0
        
        for epoch in range(local_epochs):
            for user_ids_batch, item_ids_batch, labels_batch in dataloader:
                current_user_texts = [user_texts[uid.item()] for uid in user_ids_batch]
                
                optimizer.zero_grad()
                predictions, user_linear_weight_matrix, item_embs_batch = model(user_ids_batch, item_ids_batch, current_user_texts)
                
                # 損失計算 (L_1のみ)
                loss = nn.BCELoss()(predictions.squeeze(), labels_batch)
                
                loss.backward()
                optimizer.step()
                local_loss += loss.item()

        # クライアントがサーバーにアップロードするパラメータを収集
        # 各クライアントは自身の user_joint_embedding_linear.weight をベクトル化してアップロード 
        client_linear_weights_for_graph[client_id] = model.user_joint_embedding_linear.weight.data.clone().flatten()
        
        # 各クライアントは自身の local_item_embedding.weight もアップロード
        client_local_item_weights_to_server[client_id] = model.local_item_embedding.weight.data.clone()

        print(f"  Client {client_id} local loss: {local_loss / len(dataloader):.4f}")

    # サーバーでの処理
    # ユーザー関係グラフの構築 
    user_graph_adj, sorted_client_ids_for_graph = server.build_user_relationship_graph(
        client_linear_weights_for_graph
    )
    
    # アイテム埋め込みの集約 
    server.aggregate_item_embeddings(
        client_local_item_weights_to_server, 
        user_graph_adj, 
        sorted_client_ids_for_graph
    )

    print(f"Round {round_num + 1} completed. Global item embeddings updated.")

print("Federated training completed.")

PLM embedding dimension: 384
Number of users: 100
Number of items: 50
Number of interactions: 5000
Number of clients: 10

--- Communication Round 1/10 ---
  Client 0 local loss: 0.6406
  Client 1 local loss: 0.6355
  Client 2 local loss: 0.6334
  Client 3 local loss: 0.6324
  Client 4 local loss: 0.6702
  Client 5 local loss: 0.6325
  Client 6 local loss: 0.6666
  Client 7 local loss: 0.6481
  Client 8 local loss: 0.6350
  Client 9 local loss: 0.6254
Round 1 completed. Global item embeddings updated.

--- Communication Round 2/10 ---
  Client 0 local loss: 0.6058
  Client 1 local loss: 0.5967
  Client 2 local loss: 0.6278
  Client 3 local loss: 0.6148
  Client 4 local loss: 0.6546
  Client 5 local loss: 0.6054
  Client 6 local loss: 0.6500
  Client 7 local loss: 0.6358
  Client 8 local loss: 0.6074
  Client 9 local loss: 0.6136
Round 2 completed. Global item embeddings updated.

--- Communication Round 3/10 ---
  Client 0 local loss: 0.6023
  Client 1 local loss: 0.5866
  Client 2 loca

1クライアント1ユーザに修正

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from transformers import AutoModel, AutoTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# 軽量 LLM 埋め込みモデルのロード (変更なし)
plm_model_name = "sentence-transformers/all-MiniLM-L6-v2"
plm_tokenizer = AutoTokenizer.from_pretrained(plm_model_name)
plm_model = AutoModel.from_pretrained(plm_model_name)

# PLMは学習済みモデルのため、勾配計算を無効化
for param in plm_model.parameters():
    param.requires_grad = False

plm_embedding_dim = plm_model.config.hidden_size
print(f"PLM embedding dimension: {plm_embedding_dim}")


class ClientModel(nn.Module):
    def __init__(self, num_items, item_embedding_dim, plm_model, plm_embedding_dim, joint_embedding_output_dim):
        super(ClientModel, self).__init__()
        self.plm_model = plm_model
        
        # Joint Embedding Layer の線形変換部分 (ユーザーのローカルパラメータ)
        # 論文の「trainable linear layer」[cite: 75]
        self.user_joint_embedding_linear = nn.Linear(plm_embedding_dim, joint_embedding_output_dim)
        
        # アイテム埋め込み (各クライアントのローカルパラメータとして初期化される)
        self.local_item_embedding = nn.Embedding(num_items, item_embedding_dim)

        # 予測層 (各クライアントのローカルパラメータ)
        self.prediction_layer = nn.Linear(joint_embedding_output_dim + item_embedding_dim, 1)

    def forward(self, user_ids, item_ids, user_texts_batch):
        # ユーザーのテキスト特徴をPLMで埋め込み
        encoded_input = plm_tokenizer(user_texts_batch, padding=True, truncation=True, return_tensors='pt')
        plm_output = self.plm_model(**encoded_input).last_hidden_state[:, 0, :] # [CLS]トークンの埋め込みを使用 [cite: 73]

        # Joint Embedding Layer の線形変換
        # 論文の式(3): e_u = h(v_u) = v_u W_d1xd + b [cite: 78]
        user_embedding = self.user_joint_embedding_linear(plm_output)

        # ローカルのアイテム埋め込み
        item_embedding = self.local_item_embedding(item_ids)

        # ユーザー埋め込みとアイテム埋め込みを結合
        combined_features = torch.cat((user_embedding, item_embedding), dim=1)

        # 予測
        prediction = torch.sigmoid(self.prediction_layer(combined_features))
        # 論文では「user joint embedding weights」[cite: 61] と「local item embeddings」[cite: 61] をサーバーに送信するとある。
        # ここでは、user_joint_embedding_linear.weight と local_item_embedding.weight を返す
        return prediction, self.user_joint_embedding_linear.weight, self.local_item_embedding.weight


class Server:
    def __init__(self, num_users, num_items, item_embedding_dim, joint_embedding_output_dim):
        # グローバルアイテム埋め込みは、全アイテムの埋め込み行列として管理
        self.global_item_embedding = nn.Embedding(num_items, item_embedding_dim)
        self.num_users = num_users
        self.num_items = num_items
        self.item_embedding_dim = item_embedding_dim
        self.joint_embedding_output_dim = joint_embedding_output_dim
    
    def build_user_relationship_graph(self, user_linear_weights_map):
        """
        各ユーザーのuser_joint_embedding_linear.weightからユーザー関係グラフを構築します。
        論文の式 (15) に基づいています。
        
        Args:
            user_linear_weights_map (dict): {user_id: user_joint_embedding_linear.weight.data (flattened)}
        
        Returns:
            np.ndarray: ユーザーグラフの隣接行列 (NumPy配列)
            list: グラフのノード順に対応するユーザーIDのリスト
        """
        # ユーザーIDをグラフのノードとして扱う
        sorted_user_ids = sorted(user_linear_weights_map.keys())
        if not sorted_user_ids:
            return np.zeros((0, 0)), []

        # 各ユーザーの線形層の重みベクトルを収集
        # 論文の「w_i = vec(W_i)」に相当 [cite: 105]
        user_weight_vectors = np.array([
            user_linear_weights_map[u_id].cpu().numpy() for u_id in sorted_user_ids
        ])

        # コサイン類似度で類似度行列を計算 (S_ij)
        # 論文の式(15)に相当 [cite: 106]
        similarity_matrix = cosine_similarity(user_weight_vectors)

        # ここでは簡単のため、完全な類似度グラフを使用 (S' に相当)
        # 論文の「take the top-N in the highest similarity list」 は、後のステップで実装可能 [cite: 108]
        user_graph_adj = similarity_matrix 
        
        return user_graph_adj, sorted_user_ids

    def aggregate_item_embeddings(self, user_local_item_weights, user_graph_adj, sorted_user_ids):
        """
        ユーザー関係グラフに基づいて、アイテム埋め込みをグローバルに集約します。
        論文の式 (16) と (17) に基づいています。
        
        Args:
            user_local_item_weights (dict): {user_id: local_item_embedding.weight.data (Tensor)}
            user_graph_adj (np.ndarray): ユーザーグラフの隣接行列
            sorted_user_ids (list): user_graph_adj のノード順に対応するユーザーIDのリスト
            
        Returns:
            torch.Tensor: 更新されたグローバルアイテム埋め込みの重み
        """
        if not user_local_item_weights:
            return self.global_item_embedding.weight.data

        # グラフの順序に合わせて各ユーザーのアイテム埋め込みを行列Aとしてまとめる
        # A は (num_users, num_items, item_embedding_dim)
        # 論文の「A is the round item embedding matrix, the I-th row represents the item embedding obtained from user i」に相当 
        item_embedding_matrix_A = torch.stack([
            user_local_item_weights[u_id] for u_id in sorted_user_ids
        ]) # (num_users, num_items, item_embedding_dim)

        # グラフの正規化 (S'')
        row_sums_graph = np.sum(user_graph_adj, axis=1, keepdims=True)
        row_sums_graph[row_sums_graph == 0] = 1 # ゼロ除算回避
        normalized_user_graph_adj = user_graph_adj / row_sums_graph
        
        # NumPyをPyTorchテンソルに変換
        normalized_user_graph_adj_tensor = torch.tensor(normalized_user_graph_adj, dtype=torch.float32)

        # グラフ畳み込み (R = S'' A)
        # R は (num_users, num_items, item_embedding_dim) となる
        # MatMul: (num_users, num_users) x (num_users, num_items, item_embedding_dim)
        # Einstein Summation Convention: 'ij, jkd -> ikd'
        R_tensor = torch.einsum('ij, jkd -> ikd', normalized_user_graph_adj_tensor, item_embedding_matrix_A) # 

        # グローバルアイテム埋め込みの更新 (θ_global = DR)
        # 論文の式(17)に相当 [cite: 113]
        # ここではDを全ユーザーの単純平均と解釈 (Rの0次元目を平均)
        new_global_item_embedding_weight = R_tensor.mean(dim=0) # (num_items, item_embedding_dim)

        # サーバーのグローバルアイテム埋め込みを直接更新
        self.global_item_embedding.weight.data.copy_(new_global_item_embedding_weight)
        
        return self.global_item_embedding.weight.data


# データセットの準備とクライアントへの分割 (変更)
# 1クライアント1ユーザーをシミュレート
num_users = 100
num_items = 50
num_clients = num_users # ここを修正: クライアント数とユーザー数を同じにする

user_texts = {i: f"This user likes movies about {i % 5} and enjoys {i % 3}." for i in range(num_users)}

interactions_list = []
for u_id in range(num_users):
    for i_id in range(num_items):
        if np.random.rand() > 0.7:
            interactions_list.append([u_id, i_id, 1])
        else:
            interactions_list.append([u_id, i_id, 0])

interactions = torch.tensor(interactions_list, dtype=torch.float32)

client_user_map = {} # {client_id: user_id}
client_datasets = {}
for u_id in range(num_users):
    client_id = u_id # クライアントIDをユーザーIDと同じにする NOTE: 1クライアント複数ユーザのようなケースにも対応できるようにしているが今回は、1クライアント1ユーザ.
    client_user_map[client_id] = u_id

    # 各クライアントは自身のユーザーのインタラクションデータのみを持つ
    client_interactions_indices = [i for i, (u, _, _) in enumerate(interactions_list) if u == u_id]
    if not client_interactions_indices: # インタラクションがないユーザーの場合のハンドリング
        print(f"Warning: User {u_id} has no interactions. Client {client_id} will have an empty dataset.")
        client_subset = TensorDataset(torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.float32))
    else:
        client_subset = Subset(TensorDataset(interactions[:, 0].long(), interactions[:, 1].long(), interactions[:, 2]), client_interactions_indices)

    # バッチサイズを小さくするか、ユーザーごとのインタラクション数に合わせる
    # 各ユーザーのインタラクション数が少ない場合があるので、batch_size=1でもよい
    client_datasets[client_id] = DataLoader(client_subset, batch_size=min(32, max(1, len(client_interactions_indices))), shuffle=True)

print(f"Number of users: {num_users}")
print(f"Number of items: {num_items}")
print(f"Total interactions: {len(interactions)}")
print(f"Number of clients (1 client per user): {num_clients}")


# モデルのハイパーパラメータ
item_embedding_dim = 32
joint_embedding_output_dim = 32 

# サーバーのインスタンス化
server = Server(num_users, num_items, item_embedding_dim, joint_embedding_output_dim)

# 各クライアントのモデルを辞書で保持
client_models = {}
client_optimizers = {}
for client_id in range(num_clients):
    client_models[client_id] = ClientModel(num_items, item_embedding_dim, plm_model, plm_embedding_dim, joint_embedding_output_dim)
    client_optimizers[client_id] = optim.Adam(client_models[client_id].parameters(), lr=0.001)

# 学習ループ (フェデレーテッド学習ラウンド)
num_communication_rounds = 10
local_epochs = 1 

for round_num in range(num_communication_rounds):
    print(f"\n--- Communication Round {round_num + 1}/{num_communication_rounds} ---")
    
    # サーバーからグローバルアイテム埋め込みをクライアントに配布
    # 論文のステップ「Global Distribution: Updated global project embeddings are broadcast to all clients for next-round initialization.」[cite: 63]
    for client_id in range(num_clients):
        client_models[client_id].local_item_embedding.weight.data.copy_(server.global_item_embedding.weight.data)

    user_linear_weights_for_graph = {} # {user_id: user_joint_embedding_linear.weight.data (flattened)}
    user_local_item_weights_to_server = {} # {user_id: local_item_embedding.weight.data (Tensor)}
    
    # クライアントのローカル学習
    # 論文のステップ「Local Training」[cite: 60]
    for client_id in range(num_clients):
        model = client_models[client_id]
        optimizer = client_optimizers[client_id]
        dataloader = client_datasets[client_id]
        
        model.train()
        local_loss = 0
        
        # このクライアントが担当するユーザーID
        current_user_id = client_user_map[client_id] 
        
        if len(dataloader.dataset) == 0: # インタラクションがない場合はスキップ
            print(f"  Client {client_id} (User {current_user_id}) has no interactions, skipping local training.")
            # グラフ構築のために、初期状態の重みを使用するか、ゼロベクトルを使用するか決定する必要がある
            # ここでは、訓練されなかったクライアントのために、現在のモデルの重み（グローバル初期化時と同じ）をアップロードする
            user_linear_weights_for_graph[current_user_id] = model.user_joint_embedding_linear.weight.data.clone().flatten()
            user_local_item_weights_to_server[current_user_id] = model.local_item_embedding.weight.data.clone()
            continue

        for epoch in range(local_epochs):
            for user_ids_batch, item_ids_batch, labels_batch in dataloader:
                # 1クライアント1ユーザーなので、user_ids_batchはすべて同じユーザーIDになるはず
                assert torch.all(user_ids_batch == current_user_id) 
                current_user_texts = [user_texts[uid.item()] for uid in user_ids_batch]
                
                optimizer.zero_grad()
                # predictions, user_linear_weight_matrix, item_embs_batch = model(user_ids_batch, item_ids_batch, current_user_texts)
                # ClientModelのforwardの戻り値を修正したため、それに合わせる
                predictions, user_joint_embedding_linear_weight, local_item_embedding_weight = model(user_ids_batch, item_ids_batch, current_user_texts)
                
                # 損失計算 (L_1のみ)
                # 論文の式(10) L_1(y,y^) [cite: 91]
                loss = nn.BCELoss()(predictions.squeeze(), labels_batch)
                
                loss.backward()
                optimizer.step()
                local_loss += loss.item()

        # クライアントがサーバーにアップロードするパラメータを収集
        # 論文のステップ「Parameter Uploading: Clients transmit user joint embedding weights and local item embeddings to the server.」[cite: 61]
        # 各ユーザー（クライアント）は自身の user_joint_embedding_linear.weight をベクトル化してアップロード
        user_linear_weights_for_graph[current_user_id] = user_joint_embedding_linear_weight.data.clone().flatten()
        
        # 各ユーザー（クライアント）は自身の local_item_embedding.weight もアップロード
        user_local_item_weights_to_server[current_user_id] = local_item_embedding_weight.data.clone()

        print(f"  Client {client_id} (User {current_user_id}) local loss: {local_loss / len(dataloader):.4f}")

    # サーバーでの処理
    # 論文のステップ「Graph Aggregation: The server constructs user relation graphs from text embeddings and aggregates parameters through graph convolution.」[cite: 62]
    # ユーザー関係グラフの構築
    # 論文のステップ「Build User Relaction Graph」[cite: 102, 122]
    user_graph_adj, sorted_user_ids_for_graph = server.build_user_relationship_graph(
        user_linear_weights_for_graph
    )
    
    # アイテム埋め込みの集約
    # 論文のステップ「Learn user common item embeddings with Eq.(16)」[cite: 122] と
    # 「Learn globally shared item embedding θ_global with Eq.(17)」[cite: 122]
    server.aggregate_item_embeddings(
        user_local_item_weights_to_server, 
        user_graph_adj, 
        sorted_user_ids_for_graph
    )

    print(f"Round {round_num + 1} completed. Global item embeddings updated.")

print("Federated training completed.")

/Users/kentaro.suzuki/Library/Caches/pypoetry/virtualenvs/project-UD7q69fU-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PLM embedding dimension: 384
Number of users: 100
Number of items: 50
Total interactions: 5000
Number of clients (1 client per user): 100

--- Communication Round 1/10 ---
  Client 0 (User 0) local loss: 0.7276
  Client 1 (User 1) local loss: 0.7022
  Client 2 (User 2) local loss: 0.7561
  Client 3 (User 3) local loss: 0.7084
  Client 4 (User 4) local loss: 0.7386
  Client 5 (User 5) local loss: 0.6692
  Client 6 (User 6) local loss: 0.7510
  Client 7 (User 7) local loss: 0.7375
  Client 8 (User 8) local loss: 0.7595
  Client 9 (User 9) local loss: 0.6605
  Client 10 (User 10) local loss: 0.6423
  Client 11 (User 11) local loss: 0.6982
  Client 12 (User 12) local loss: 0.7081
  Client 13 (User 13) local loss: 0.7375
  Client 14 (User 14) local loss: 0.6891
  Client 15 (User 15) local loss: 0.6695
  Client 16 (User 16) local loss: 0.7670
  Client 17 (User 17) local loss: 0.7243
  Client 18 (User 18) local loss: 0.6946
  Client 19 (User 19) local loss: 0.7323
  Client 20 (User 20) local 

KeyboardInterrupt: 